In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys, json, pickle
sys.path.append('../scripts/')
import land_cover_analysis as lca
import land_cover_visualisation as lcv
import land_cover_models as lcm
import loadpaths 

path_dict = loadpaths.loadpaths()  # load paths (see content/README_datapaths.md)


## Loading existing models
If you want to use the CNN models from our paper, you can download these [here](https://drive.google.com/drive/folders/1nEnIWDvWcLVzSE6yViv93I4klY2WzdDo?usp=sharing).

In the following, we will load these (also applicable to other models).

There are two ways of loading models: loading the full model (`.data`), or loading the resnet weights only (`.pth`). The `lcm.load_model_auto()` function does this automatically, by recognising the file extension. To load the full models provided, you need an older version of torch/lightning (see `envs/geo_exactbuilds.txt`). The weights-only models can also be loaded with latest torch/lightning, and are equivalent.

In [18]:
folder_model = '/home/thijs/Google Drive/peak district/models/2024-03-22/saved_cnn_models'
# file_name = 'main_LCU_2023-04-24-1259_FULL.data'
file_name = 'main_LCU_2023-04-24-1259.pth'

In [19]:
LCU = lcm.load_model_auto(folder=folder_model, filename=file_name)

cross_entropy loss is used.


In [20]:
LCU.dict_training_details

{'class_name_list': ['NO CLASS',
  'Wood and Forest Land',
  'Moor and Heath Land',
  'Agro-Pastoral Land']}

## Loading the train/test patch data set
To train and test the model, you can use the data set of image patches (= 512x 512 pixels) from our paper [available here](https://cord.cranfield.ac.uk/articles/dataset/Very_high_resolution_aerial_photography_and_annotated_land_cover_data_of_the_Peak_District_National_Park/24221314). These can be loaded as follows:

In [15]:
assert os.path.exists(path_dict['im_patches']), 'Set path of image patches folder (from data repo link above) in datapaths.json' 

tile_patch_train_test_split_dict_path='../content/evaluation_sample_50tiles/train_test_split_80tiles_2023-03-22-2131.pkl'
if tile_patch_train_test_split_dict_path is not None:
    with open(tile_patch_train_test_split_dict_path, 'rb') as f:
        dict_tile_patches = pickle.load(f)
        tile_patch_train = dict_tile_patches['train']
        tile_patch_test = dict_tile_patches['test']
else:
    tile_patch_train = None
    tile_patch_test = None

mapping_dicts_list = [
          '../content/label_mapping_dicts/label_mapping_dict__all_relevant_subclasses__2023-04-20-1540.pkl',
        '../content/label_mapping_dicts/label_mapping_dict__C_subclasses_only__2023-04-20-1540.pkl',
        '../content/label_mapping_dicts/label_mapping_dict__D_subclasses_only__2023-04-20-1540.pkl',
        '../content/label_mapping_dicts/label_mapping_dict__E_subclasses_and_F3d_only__2023-04-20-1541.pkl',
        '../content/label_mapping_dicts/label_mapping_dict__main_categories_F3inDE_noFGH__2023-04-21-1315.pkl'
                         ]
path_mapping_dict = mapping_dicts_list[3]

train_ds = lcm.DataSetPatches(im_dir=path_dict['im_patches'], mask_dir=None, 
                                mask_suffix='_lc_2022_detailed_mask.npy', 
                                mask_dir_name='masks_python_all',
                                #   list_tile_names=dict_tile_names_sample['train'],
                                list_tile_patches_use=tile_patch_train,
                                preprocessing_func=LCU.preprocessing_func,
                                shuffle_order_patches=True, relabel_masks=True,
                                subsample_patches=False, path_mapping_dict=path_mapping_dict,
                                random_transform_data=False)

No mask directory provided. Will use masks_python_all/ in image parent directory instead.
Only using patches that are in tile_patches list (of length 720).
Patches ordered randomly
Loaded 719 patches
Loaded label_mapping_dict__E_subclasses_and_F3d_only__2023-04-20-1541.pkl to map labels


In [16]:
train_ds.class_name_list

['NO CLASS',
 'Cultivated Land',
 'Improved Pasture',
 'Rough Pasture',
 'Wetland, Wet Grassland and Rush Pasture']

In [17]:
for path_mapping_dict in mapping_dicts_list:
    tmp_path_dict = pickle.load(open(path_mapping_dict, 'rb'))
    n_classes = len(tmp_path_dict['dict_new_names'])
    print(path_mapping_dict, n_classes)
    

../content/label_mapping_dicts/label_mapping_dict__all_relevant_subclasses__2023-04-20-1540.pkl 14
../content/label_mapping_dicts/label_mapping_dict__C_subclasses_only__2023-04-20-1540.pkl 5
../content/label_mapping_dicts/label_mapping_dict__D_subclasses_only__2023-04-20-1540.pkl 7
../content/label_mapping_dicts/label_mapping_dict__E_subclasses_and_F3d_only__2023-04-20-1541.pkl 5
../content/label_mapping_dicts/label_mapping_dict__main_categories_F3inDE_noFGH__2023-04-21-1315.pkl 4
